<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
     <h3> Objective</h3> <br>
    <b style = "color:#000000"><i>Rotate Access Key for an IAM user using unSkript actions</i></b>
</div>
<br>
</center>

<center><h2><u>AWS Access Key Rotation</u></h2></center>

# Steps Overview
1)[ Create AWS Access Key](#1)<br>
2)[ Update AWS Access Key](#2)<br>
3)[ List AWS Access Keys](#3)<br>
4)[ Delete AWS Access Key](#4)<br>

<h3><a id='1'>Create AWS Access Keys</a></h3>
Using unSkript's AWS Create Access Key action we will create a new Access Key for the user. 

>Action takes the following parameters: `aws_username`


In [11]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint


from beartype import beartype
@beartype
def aws_create_access_key_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_create_access_key(
    handle,
    aws_username: str
) -> Dict:
    """aws_create_access_key creates a new access key for the given user.
            :type handle: object
            :param handle: Object returned from Task Validate

            :type aws_username: str
            :param aws_username: Username of the IAM user to be looked up

            :rtype: Result Dictionary of result
    """
    iamClient = handle.client('iam')
    result = iamClient.create_access_key(UserName=aws_username)
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_username": "aws_username"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_access_key, lego_printer=aws_create_access_key_printer, hdl=hdl, args=args)

<h3><a id='2'>Update AWS Access Key</a></h3>
Using the AWS Update Access Key action we will update the status of the old Access Key to <b>"Inactive"</b>. This step is required to delete the old access key as one user cannot have 2 Access Keys. 

>This action takes the following parameters: `aws_username`, `aws_access_key_id` and `status`

In [12]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict,List
import pprint


from beartype import beartype
@beartype
def aws_update_access_key_printer(output):
    if output is None:
        return
    pprint.pprint("Access Key status successfully changed")
    pprint.pprint(output)


@beartype
def aws_update_access_key(
    handle,
    aws_username: str,
    aws_access_key_id: str,
    status: str
) -> Dict:
    """aws_update_access_key updates the status of an access key to Inactive/Active
                    :type handle: object
                    :param handle: Object returned from Task Validate

                    :type aws_username: str
                    :param aws_username: Username of the IAM user to be looked up

                    :type aws_access_key_id: str
                    :param aws_access_key_id: Old Access Key ID of the user of which the status needs to be updated

                    :rtype: Result Dictionary of result
    """
    iamClient = handle.client('iam')
    result = iamClient.update_access_key(UserName=aws_username, AccessKeyId=aws_access_key_id, Status=status)
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_access_key_id": "aws_access_key_id",
    "aws_username": "aws_username",
    "status": "status"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_update_access_key, lego_printer=aws_update_access_key_printer, hdl=hdl, args=args)

<h3><a id='3'>List AWS Access Keys</a></h3>
Here we will list all the Access Keys assigned to the given IAM user. The new Access Key should have the "Active "status and the old one should have should be "Inactive".

>This action takes the following parameters: `aws_username`

In [3]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint


from beartype import beartype
@beartype
def aws_list_access_keys_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_access_keys(
        handle,
        aws_username: str
) -> Dict:
    """aws_list_access_keys lists all the access keys for a user

                :type handle: object
                :param handle: Object returned from Task Validate

                :type aws_username: str
                :param aws_username: Username of the IAM user to be looked up

                :rtype: Result Dictionary of result
    """
    iamClient = handle.client('iam')
    result = iamClient.list_access_keys(UserName=aws_username)
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_username": "aws_username"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_access_keys, lego_printer=aws_list_access_keys_printer, hdl=hdl, args=args)

<h3><a id='4'>Delete AWS Access Key</a></h3>
Finally, we will delete the the old (Inactive) Access Key for the IAM User

>This action takes the following parameters: `aws_username` and `aws_access_key_id`

In [14]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict,List
import pprint


from beartype import beartype
@beartype
def aws_delete_access_key_printer(output):
    if output is None:
        return
    pprint.pprint("Access Key successfully deleted")
    pprint.pprint(output)


@beartype
def aws_delete_access_key(
    handle,
    aws_username: str,
    aws_access_key_id: str,
) -> Dict:
    """aws_delete_access_key deleted the given access key.
                    :type handle: object
                    :param handle: Object returned from Task Validate

                    :type aws_username: str
                    :param aws_username: Username of the IAM user to be looked up

                    :type aws_access_key_id: str
                    :param aws_access_key_id: Old Access Key ID of the user which needs to be deleted

                    :rtype: Result Status Dictionary of result
    """
    iamClient = handle.client('iam')
    result = iamClient.delete_access_key(UserName=aws_username, AccessKeyId=aws_access_key_id)
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_access_key_id": "aws_access_key_id",
    "aws_username": "aws_username"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_access_key, lego_printer=aws_delete_access_key_printer, hdl=hdl, args=args)

<h3>Conclusion</h3>
<p>In this Runbook, we were able to perform AWS Access Key rotation for a given IAM user by using unSkript's AWS actions. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io">us.app.unskript.io</a></p>